### Importing Libr

In [106]:
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
from selenium import webdriver
import pandas as pd # Импортируем библиотеку Pandas
from fake_useragent import UserAgent
from selenium.webdriver.chrome.options import Options
from time import sleep
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from datetime import datetime


### Import links to scrape

In [107]:

links = pd.read_excel('Database_new.xlsx')
database = pd.read_excel("Database_new.xlsx")
display(links.info())

apple_links = links[(links['Brand']=='Goldapple') & (links['Link'].notna())]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1206 entries, 0 to 1205
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            1206 non-null   object
 1   Brand         1206 non-null   object
 2   Product       1206 non-null   object
 3   Concept code  1206 non-null   object
 4   Category      1206 non-null   object
 5   Price Shelf   1206 non-null   object
 6   Sector        1206 non-null   object
 7   Segment       1205 non-null   object
 8   Link          1204 non-null   object
dtypes: object(9)
memory usage: 84.9+ KB


None

'Avon-'

(87, 9)

'Faberlic-'

...

(6, 9)

'Yves-'

(7, 9)

'Herb-'

(6, 9)

### Parser

In [118]:

apple_dict = apple_links[['Id', 'Link']].set_index('Id').T.to_dict('Link')
apple_links_new = []

for key, value in apple_dict.items():
    list_ = [key,value]
    apple_links_new.append(list_)

/var/folders/10/0pwhf9w54g7cp280nnbyhsfw0000gn/T/ipykernel_66155/1912706036.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  apple_dict = apple_links[['Id', 'Link']].set_index('Id').T.to_dict('Link')


In [119]:
apple_df = pd.DataFrame(columns=["Id", "Product", "Standart_price", "Merch_price", "Link", "Date"])
apple_database = pd.read_excel("apple_pars.xlsx")

class GA(webdriver.Chrome):
    def __init__(self, driver_path = "Scraping/"):
        self.driver_path = driver_path
        os.environ['PATH'] = self.driver_path
        super(GA, self).__init__()
        self.maximize_window()
        self.implicitly_wait(20)
        
    def ga_scrape_page(self):
        
        global apple_df, apple_database
        
        try:
        
            for link in apple_links_new:
                
                self.get(link[1][0])
                sleep(5)
                
                Id = link[0]
                results_a = []
                results_b = []
                results_c = []

                prod = []
                price_stand = []
                price_merch = []
                current_link = link[1][0]

                content = self.page_source
                soup = BeautifulSoup(content) 
                
                data_a = soup.find('div', class_ = 'oWdx+ BHvhN')
                if data_a not in results_a:
                    if data_a:
                        if data_a.text.replace("\n", "") != ' 0\xa0₽ ':
                            results_a.append(data_a.text)

            
                data_b = soup.find('div', class_ = 'oWdx+ _74opb HCpOL BHvhN')
                if data_b not in results_b:
                    if data_b:
                        if data_b.text.replace("\n", "") != ' 0\xa0₽ ':
                            results_b.append(data_b.text)
                    else:
                        results_b = results_a
                    
                data_c = soup.find('h1', class_ = 'PdKWR')
                if data_c not in results_c:
                    if data_c:
                        results_c.append(data_c.text)
                    
        
                for pm in results_a:
                    price_merch.append(pm.replace("\n", "").replace("  ", ""))

                for ps in results_b:
                    price_stand.append(ps.replace("\n", "").replace("  ", ""))

                for pr in results_c:
                    prod.append(pr.replace("\n", "").replace("  ", ""))
            
                if results_a:    
                    apple = pd.DataFrame({"Id": Id, "Product": prod, "Standart_price": price_stand[0], "Merch_price": price_merch, "Link":current_link})
                    apple_df = apple_df.append(apple, ignore_index=True)
                    apple_df["Date"] = datetime.today().strftime('%Y-%m-%d')
                    apple_df = apple_df.drop_duplicates(subset=["Product", "Link", "Date"])
                    apple_database = pd.concat([apple_database, apple_df] , axis=0, ignore_index=True)
                    apple_database = apple_database.drop_duplicates(subset=["Product", "Link", "Date"])
                    apple_database.to_excel("apple_pars.xlsx", sheet_name="Sheet1", index=False)
                
        except:
            
            last_id = apple_df.tail(1).Id.values
            last_index = 0

            for i in apple_links_new:
                last_index = last_index + 1
                if i[0] == last_id:
                    last_index = last_index - 1
                    break
            
            for link in apple_links_new[last_index:]:
                
                self.get(link[1][0])
                sleep(5)
                
                Id = link[0]
                results_a = []
                results_b = []
                results_c = []

                prod = []
                price_stand = []
                price_merch = []
                current_link = link[1][0]

                content = self.page_source
                soup = BeautifulSoup(content) 
                
                data_a = soup.find('div', class_ = 'oWdx+ BHvhN')
                if data_a not in results_a:
                    if data_a:
                        if data_a.text.replace("\n", "") != ' 0\xa0₽ ':
                            results_a.append(data_a.text)

            
                data_b = soup.find('div', class_ = 'oWdx+ _74opb HCpOL BHvhN')
                if data_b not in results_b:
                    if data_b:
                        if data_b.text.replace("\n", "") != ' 0\xa0₽ ':
                            results_b.append(data_b.text)
                    else:
                        results_b = results_a
                    
                data_c = soup.find('h1', class_ = 'PdKWR')
                if data_c not in results_c:
                    if data_c:
                        results_c.append(data_c.text)
                    
        
                for pm in results_a:
                    price_merch.append(pm.replace("\n", "").replace("  ", ""))

                for ps in results_b:
                    price_stand.append(ps.replace("\n", "").replace("  ", ""))

                for pr in results_c:
                    prod.append(pr.replace("\n", "").replace("  ", ""))
            
                if results_a:    
                    apple = pd.DataFrame({"Id": Id, "Product": prod, "Standart_price": price_stand[0], "Merch_price": price_merch, "Link":current_link})
                    apple_df = apple_df.append(apple, ignore_index=True)
                    apple_df["Date"] = datetime.today().strftime('%Y-%m-%d')
                    apple_df = apple_df.drop_duplicates(subset=["Product", "Link", "Date"])
                    apple_database = pd.concat([apple_database, apple_df] , axis=0, ignore_index=True)
                    apple_database = apple_database.drop_duplicates(subset=["Product", "Link", "Date"])
                    apple_database.to_excel("apple_pars.xlsx", sheet_name="Sheet1", index=False)
                    
inst_ga = GA()
inst_ga.ga_scrape_page       